<a href="https://colab.research.google.com/github/pinggu95/deep_dive_AI/blob/main/63%EC%9D%BC%EC%B0%A8/63%EC%9D%BC%EC%B0%A8_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 벡터스토어 기반 검색기(VectorStore-backed Retriever)

**VectorStore 지원 검색기** 는 vector store를 사용하여 문서를 검색하는 retriever입니다.

Vector store에 구현된 **유사도 검색(similarity search)** 이나 **MMR** 과 같은 검색 메서드를 사용하여 vector store 내의 텍스트를 쿼리합니다.


아래의 코드를 실행하여 VectorStore 를 생성합니다.

In [1]:
!pip install python-dotenv

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
!pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-Retriever")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [7]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 67.2 MB/s eta 0:00:00


In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader("/content/appendix-keywords.txt")

# 문서를 로드합니다.
documents = loader.load()

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 300이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 로드된 문서를 분할합니다.
split_docs = text_splitter.split_documents(documents)

# OpenAI 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# 분할된 텍스트와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다.
db = FAISS.from_documents(split_docs, embeddings)

### VectorStore에서 VectorStoreRetriever 초기화(as_retriever)

`as_retriever` 메서드는 VectorStore 객체를 기반으로 VectorStoreRetriever를 초기화하고 반환합니다. 이 메서드를 통해 다양한 검색 옵션을 설정하여 사용자의 요구에 맞는 문서 검색을 수행할 수 있습니다.

**매개변수(parameters)**

- `**kwargs`: 검색 함수에 전달할 키워드 인자
  - `search_type`: 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
  - `search_kwargs`: 추가 검색 옵션
    - `k`: 반환할 문서 수 (기본값: 4)
    - `score_threshold`: similarity_score_threshold 검색의 최소 유사도 임계값
    - `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
    - `lambda_mult`: MMR 결과의 다양성 조절 (0-1 사이, 기본값: 0.5)
    - `filter`: 문서 메타데이터 기반 필터링

**반환값(return)**

- `VectorStoreRetriever`: 초기화된 VectorStoreRetriever 객체

**참고**

- 다양한 검색 전략 구현 가능 (유사도, MMR, 임계값 기반)
- MMR (Maximal Marginal Relevance) 알고리즘으로 검색 결과의 다양성 조절 가능
- 메타데이터 필터링으로 특정 조건의 문서만 검색 가능
- `tags` 매개변수를 통해 검색기에 태그 추가 가능

**주의사항**

- `search_type`과 `search_kwargs`의 적절한 조합 필요
- MMR 사용 시 `fetch_k`와 `k` 값의 균형 조절 필요
- `score_threshold` 설정 시 너무 높은 값은 검색 결과가 없을 수 있음
- 필터 사용 시 데이터셋의 메타데이터 구조 정확히 파악 필요
- `lambda_mult` 값이 0에 가까울수록 다양성이 높아지고, 1에 가까울수록 유사성이 높아짐

In [9]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당
retriever = db.as_retriever()

### Retriever의 invoke()

`invoke` 메서드는 Retriever의 주요 진입점으로, 관련 문서를 검색하는 데 사용됩니다. 이 메서드는 동기적으로 Retriever를 호출하여 주어진 쿼리에 대한 관련 문서를 반환합니다.

**매개변수(parameters)**

- `input`: 검색 쿼리 문자열
- `config`: Retriever 구성 (Optional[RunnableConfig])
- `**kwargs`: Retriever에 전달할 추가 인자

**반환값(return)**

- `List[Document]`: 관련 문서 목록

In [10]:
# 관련 문서를 검색
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec


### Max Marginal Relevance (MMR)

`MMR(Maximal Marginal Relevance)` 방식은 쿼리에 대한 관련 항목을 검색할 때 검색된 문서의 **중복** 을 피하는 방법 중 하나입니다.

단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 쿼리에 대한 **문서의 관련성** 과 이미 선택된 **문서들과의 차별성을 동시에 고려** 합니다.


- `search_type` 매개변수를 `"mmr"` 로 설정하여 **MMR(Maximal Marginal Relevance)** 검색 알고리즘을 사용합니다.
- `k`: 반환할 문서 수 (기본값: 4)
- `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
- `lambda_mult`: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5, 0: 유사도 점수만 고려, 1: 다양성만 고려)

In [11]:
# MMR(Maximal Marginal Relevance) 검색 유형을 지정
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10, "lambda_mult": 0.6}
)

# 관련 문서를 검색합니다.
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


### 유사도 점수 임계값 검색(similarity_score_threshold)

유사도 점수 임계값을 설정하고 해당 임계값 이상의 점수를 가진 문서만 반환하는 검색 방법을 설정할 수 있습니다.

임계값을 적절히 설정함으로써 **관련성이 낮은 문서를 필터링** 하고, 질의와 **가장 유사한 문서만 선별** 할 수 있습니다.

- `search_type` 매개변수를 `"similarity_score_threshold"` 로 설정하여 유사도 점수 임계값을 기준으로 검색을 수행합니다.

- `search_kwargs` 매개변수에 `{"score_threshold": 0.8}`를 전달하여 유사도 점수 임계값을 0.8로 설정합니다. 이는 검색 결과의 **유사도 점수가 0.8 이상인 문서만 반환됨** 을 의미합니다.


In [12]:
retriever = db.as_retriever(
    # 검색 유형을 "similarity_score_threshold 으로 설정
    search_type="similarity_score_threshold",
    # 임계값 설정
    search_kwargs={"score_threshold": 0.8},
)

# 관련 문서를 검색
for doc in retriever.invoke("Word2Vec 은 무엇인가요?"):
    print(doc.page_content)
    print("=========================================================")

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


### top_k 설정

검색 시 사용할 `k` 와 같은 검색 키워드 인자(kwargs)를 지정할 수 있습니다.

`k` 매개변수는 검색 결과에서 반환할 상위 결과의 개수를 나타냅니다.

- `search_kwargs`에서 `k` 매개변수를 1로 설정하여 검색 결과로 반환할 문서의 수를 지정합니다.


In [13]:
# k 설정
retriever = db.as_retriever(search_kwargs={"k": 1})

# 관련 문서를 검색
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token


## 동적 설정(Configurable)

- 검색 설정을 동적으로 조정하기 위해 `ConfigurableField` 를 사용합니다.
- `ConfigurableField` 는 검색 매개변수의 고유 식별자, 이름, 설명을 설정하는 역할을 합니다.
- 검색 설정을 조정하기 위해 `config` 매개변수를 사용하여 검색 설정을 지정합니다.
- 검색 설정은 `config` 매개변수에 전달된 딕셔너리의 `configurable` 키에 저장됩니다.
- 검색 설정은 검색 쿼리와 함께 전달되며, 검색 쿼리에 따라 동적으로 조정됩니다.

In [14]:
from langchain_core.runnables import ConfigurableField

# k 설정
retriever = db.as_retriever(search_kwargs={"k": 1}).configurable_fields(
    search_type=ConfigurableField(
        id="search_type",
        name="Search Type",
        description="The search type to use",
    ),
    search_kwargs=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정
        id="search_kwargs",
        # 검색 매개변수의 이름을 설정
        name="Search Kwargs",
        # 검색 매개변수에 대한 설명을 작성
        description="The search kwargs to use",
    ),
)

아래는 동적 검색설정을 적용한 예시입니다.

In [15]:
# 검색 설정을 지정. Faiss 검색에서 k=3로 설정하여 가장 유사한 문서 3개를 반환
config = {"configurable": {"search_kwargs": {"k": 3}}}

# 관련 문서를 검색
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?", config=config)

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


In [16]:
# 검색 설정을 지정. score_threshold 0.8 이상의 점수를 가진 문서만 반환
config = {
    "configurable": {
        "search_type": "similarity_score_threshold",
        "search_kwargs": {
            "score_threshold": 0.8,
        },
    }
}

# 관련 문서를 검색
docs = retriever.invoke("Word2Vec 은 무엇인가요?", config=config)

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


In [17]:
# 검색 설정을 지정. mmr 검색 설정.
config = {
    "configurable": {
        "search_type": "mmr",
        "search_kwargs": {"k": 2, "fetch_k": 10, "lambda_mult": 0.6},
    }
}

# 관련 문서를 검색
docs = retriever.invoke("Word2Vec 은 무엇인가요?", config=config)

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec


## Upstage 임베딩과 같이 Query & Passage embedding model 이 분리된 경우

기본 retriever는 쿼리와 문서에 대해 동일한 임베딩 모델을 사용합니다.

하지만 쿼리와 문서에 대해 서로 다른 임베딩 모델을 사용하는 경우가 있습니다.

이러한 경우에는 쿼리 임베딩 모델을 사용하여 쿼리를 임베딩하고, 문서 임베딩 모델을 사용하여 문서를 임베딩합니다.

이렇게 하면 쿼리와 문서에 대해 서로 다른 임베딩 모델을 사용할 수 있습니다.

In [20]:
!pip install langchain-upstage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.55.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [27]:
import os
os.environ["UPSTAGE_API_KEY"] = "upstage API"

In [28]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_upstage import UpstageEmbeddings

# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader("/content/appendix-keywords.txt")

# 문서를 로드합니다.
documents = loader.load()

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 300이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 로드된 문서를 분할합니다.
split_docs = text_splitter.split_documents(documents)

# Upstage 임베딩을 생성합니다. 문서용 모델을 사용합니다.
doc_embedder = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 분할된 텍스트와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다.
db = FAISS.from_documents(split_docs, doc_embedder)

아래는 쿼리용 Upstage 임베딩을 생성하고, 쿼리 문장을 벡터로 변환하여 벡터 유사도 검색을 수행하는 예시입니다.

In [29]:
# 쿼리용 Upstage 임베딩을 생성합니다. 쿼리용 모델을 사용합니다.
query_embedder = UpstageEmbeddings(model="solar-embedding-1-large-query")

# 쿼리 문장을 벡터로 변환합니다.
query_vector = query_embedder.embed_query("임베딩(Embedding)은 무엇인가요?")

# 벡터 유사도 검색을 수행하여 가장 유사한 2개의 문서를 반환합니다.
db.similarity_search_by_vector(query_vector, k=2)

[Document(id='1c977f36-7661-4359-abb5-069a07ddaf18', metadata={'source': '/content/appendix-keywords.txt'}, page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken'),
 Document(id='d23c354c-f413-4079-9e6f-081bdcdfb316', metadata={'source': '/content/appendix-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)')]

# 문맥 압축 검색기(ContextualCompressionRetriever)

검색 시스템에서 직면하는 어려움 중 하나는 데이터를 시스템에 수집할 때 어떤 특정 질의를 처리해야 할지 미리 알 수 없다는 점입니다.

이는 질의와 가장 관련성이 높은 정보가 많은 양의 무관한 텍스트를 포함한 문서에 묻혀 있을 수 있음을 의미합니다.

이러한 전체 문서를 애플리케이션에 전달하면 더 비용이 많이 드는 LLM 호출과 품질이 낮은 응답으로 이어질 수 있습니다.

`ContextualCompressionRetriever` 은 이 문제를 해결하기 위해 고안되었습니다.

아이디어는 간단합니다. 검색된 문서를 그대로 즉시 반환하는 대신, 주어진 질의의 맥락을 사용하여 문서를 압축함으로써 관련 정보만 반환되도록 할 수 있습니다.

여기서 "압축"은 개별 문서의 내용을 압축하는 것과 문서를 전체적으로 필터링하는 것 모두를 의미합니다.

`ContextualCompressionRetriever` 는 질의를 base retriever에 전달하고, 초기 문서를 가져와 Document Compressor를 통과시킵니다.

Document Compressor는 문서 목록을 가져와 문서의 내용을 줄이거나 문서를 완전히 삭제하여 목록을 축소합니다.

> 출처: https://drive.google.com/uc?id=1CtNgWODXZudxAWSRiWgSGEoTNrUFT98v

![](./images/01-Contextual-Compression.jpeg)


In [30]:
# 패키지 업데이트
!pip install -qU langchain-teddynote

`pretty_print_docs` 함수는 문서 리스트를 예쁘게 출력하는 헬퍼 함수입니다.


In [31]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"문서 {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 기본 Retriever 설정

간단한 벡터 스토어 retriever를 초기화하고 텍스트 문서를 청크 단위로 저장하는 것부터 시작해 보겠습니다.

예시 질문을 던졌을 때, retriever는 관련 있는 문서 1~2개와 관련 없는 문서 몇 개를 반환하는 것을 확인할 수 있습니다.

In [33]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# TextLoader를 사용하여 "appendix-keywords.txt" 파일에서 문서를 로드합니다.
loader = TextLoader("/content/appendix-keywords.txt")

# CharacterTextSplitter를 사용하여 문서를 청크 크기 300자와 청크 간 중복 0으로 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

# OpenAIEmbeddings를 사용하여 FAISS 벡터 저장소를 생성하고 검색기로 변환합니다.
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

# 쿼리에 질문을 정의하고 관련 문서를 검색합니다.
docs = retriever.invoke("Semantic Search 에 대해서 알려줘.")

# 검색된 문서를 예쁘게 출력합니다.
pretty_print_docs(docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

## 맥락적 압축(ContextualCompression)

`LLMChainExtractor` 를 활용하여 생성한 `DocumentCompressor` 를 retriever 에 적용한 것이 바로 `ContextualCompressionRetriever` 입니다.

In [34]:
from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

# from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  # OpenAI 언어 모델 초기화

# LLM을 사용하여 문서 압축기 생성
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    # 문서 압축기와 리트리버를 사용하여 컨텍스트 압축 리트리버 생성
    base_compressor=compressor,
    base_retriever=retriever,
)

pretty_print_docs(retriever.invoke("Semantic Search 에 대해서 알려줘."))

print("=========================================================")
print("============== LLMChainExtractor 적용 후 ==================")

compressed_docs = (
    compression_retriever.invoke(  # 컨텍스트 압축 리트리버를 사용하여 관련 문서 검색
        "Semantic Search 에 대해서 알려줘."
    )
)
pretty_print_docs(compressed_docs)  # 검색된 문서를 예쁘게 출력

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

## LLM 을 활용한 문서 필터링

### `LLMChainFilter`

`LLMChainFilter`는 초기에 검색된 문서 중 어떤 문서를 필터링하고 어떤 문서를 반환할지 결정하기 위해 LLM 체인을 사용하는 보다 단순하지만 강력한 압축기입니다.

이 필터는 문서 내용을 변경(압축)하지 않고 문서를 **선택적으로 반환** 합니다.


In [35]:
from langchain_teddynote.document_compressors import LLMChainFilter

# LLM을 사용하여 LLMChainFilter 객체를 생성합니다.
_filter = LLMChainFilter.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    # LLMChainFilter와 retriever를 사용하여 ContextualCompressionRetriever 객체를 생성합니다.
    base_compressor=_filter,
    base_retriever=retriever,
)

compressed_docs = compression_retriever.invoke(
    # 쿼리
    "Semantic Search 에 대해서 알려줘."
)
pretty_print_docs(compressed_docs)  # 압축된 문서를 예쁘게 출력합니다.

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


### `EmbeddingsFilter`

각각의 검색된 문서에 대해 추가적인 LLM 호출을 수행하는 것은 비용이 많이 들고 속도가 느립니다.

`EmbeddingsFilter`는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공합니다.

이를 통해 검색 결과의 관련성을 유지하면서도 계산 비용과 시간을 절약할 수 있습니다.


`EmbeddingsFilter` 와 `ContextualCompressionRetriever` 를 사용하여 관련 문서를 압축하고 검색하는 과정입니다.

- `EmbeddingsFilter` 를 사용하여 지정된 **유사도 임계값(0.86)** 이상인 문서를 필터링 합니다.

In [36]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# 유사도 임계값이 0.76인 EmbeddingsFilter 객체를 생성합니다.
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

# 기본 압축기로 embeddings_filter를, 기본 검색기로 retriever를 사용하여 ContextualCompressionRetriever 객체를 생성합니다.
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

# ContextualCompressionRetriever 객체를 사용하여 관련 문서를 검색합니다.
compressed_docs = compression_retriever.invoke(
    # 쿼리
    "Semantic Search 에 대해서 알려줘."
)
# 압축된 문서를 예쁘게 출력합니다.
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


## 파이프라인 생성(압축기+문서 변환기)

`DocumentCompressorPipeline` 을 사용하면 여러 compressor를 순차적으로 결합할 수 있습니다.

Compressor와 함께 `BaseDocumentTransformer`를 파이프라인에 추가할 수 있는데, 이는 맥락적 압축을 수행하지 않고 단순히 문서 집합에 대한 변환을 수행합니다.

예를 들어, `TextSplitter`는 문서를 더 작은 조각으로 분할하기 위해 document transformer로 사용될 수 있으며, `EmbeddingsRedundantFilter`는 문서 간의 임베딩 유사성(기본값: 0.95 유사도 이상을 중복 문서로 간주) 을 기반으로 중복 문서를 필터링하는 데 사용될 수 있습니다.

아래에서는 먼저 문서를 더 작은 청크로 분할한 다음, 중복 문서를 제거하고, 쿼리와의 관련성을 기준으로 필터링하여 compressor pipeline을 생성합니다.


In [37]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

# 문자 기반 텍스트 분할기를 생성하고, 청크 크기를 300으로, 청크 간 중복을 0으로 설정합니다.
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 임베딩을 사용하여 중복 필터를 생성합니다.
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 임베딩을 사용하여 관련성 필터를 생성하고, 유사도 임계값을 0.86으로 설정합니다.
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

pipeline_compressor = DocumentCompressorPipeline(
    # 문서 압축 파이프라인을 생성하고, 분할기, 중복 필터, 관련성 필터, LLMChainExtractor를 변환기로 설정합니다.
    transformers=[
        splitter,
        redundant_filter,
        relevant_filter,
        LLMChainExtractor.from_llm(llm),
    ]
)

`ContextualCompressionRetriever`를 초기화하며, `base_compressor`로 `pipeline_compressor`를, `base_retriever`로 `retriever`를 사용합니다.


In [38]:
compression_retriever = ContextualCompressionRetriever(
    # 기본 압축기로 pipeline_compressor를 사용하고, 기본 검색기로 retriever를 사용하여 ContextualCompressionRetriever를 초기화합니다.
    base_compressor=pipeline_compressor,
    base_retriever=retriever,
)

compressed_docs = compression_retriever.invoke(
    # 쿼리
    "Semantic Search 에 대해서 알려줘."
)
# 압축된 문서를 예쁘게 출력합니다.
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.


# 앙상블 검색기(Ensemble Retriever)

`EnsembleRetriever`는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능입니다. 이 검색기는 다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

**주요 특징**
1. 여러 검색기 통합: 다양한 유형의 검색기를 입력으로 받아 결과를 결합합니다.
2. 결과 재순위화: [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘을 사용하여 결과의 순위를 조정합니다.
3. 하이브리드 검색: 주로 `sparse retriever`(예: BM25)와 `dense retriever`(예: 임베딩 유사도)를 결합하여 사용합니다.

**장점**
- Sparse retriever: 키워드 기반 검색에 효과적
- Dense retriever: 의미적 유사성 기반 검색에 효과적

이러한 상호 보완적인 특성으로 인해 `EnsembleRetriever`는 다양한 검색 시나리오에서 향상된 성능을 제공할 수 있습니다.

자세한 내용은 [LangChain 공식 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)를 참조하세요.


- `EnsembleRetriever`를 초기화하여 `BM25Retriever`와 `FAISS` 검색기를 결합합니다. 각 검색기의 가중치를 설정됩니다.


In [39]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 샘플 문서 리스트
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]


# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_texts(
    doc_list,
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

embedding = OpenAIEmbeddings()  # OpenAI 임베딩을 사용합니다.
faiss_vectorstore = FAISS.from_texts(
    doc_list,
    embedding,
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

`ensemble_retriever` 객체의 `get_relevant_documents()` 메서드를 호출하여 관련성 높은 문서를 검색합니다.


In [40]:
# 검색 결과 문서를 가져옵니다.
query = "my favorite fruit is apple"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apples

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apples



In [41]:
# 검색 결과 문서를 가져옵니다.
query = "Apple company makes my favorite iphone"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apple's iphone

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apple's iphone



## 런타임 Config 변경

런타임에서도 retriever 의 속성을 변경할 수 있습니다. 이는 `ConfigurableField` 클래스를 사용하여 가능합니다.

- `weights` 매개변수를 `ConfigurableField` 객체로 정의합니다.
  - 필드의 ID는 "ensemble_weights"로 설정합니다.


In [42]:
from langchain_core.runnables import ConfigurableField


ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, faiss_retriever],
).configurable_fields(
    weights=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="ensemble_weights",
        # 검색 매개변수의 이름을 설정합니다.
        name="Ensemble Weights",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="Ensemble Weights",
    )
)

- 검색 시 `config` 매개변수를 통해 검색 설정을 지정합니다.
  - `ensemble_weights` 옵션의 가중치를 [1, 0]으로 설정하여 **모든 검색 결과의 가중치가 BM25 retriever 에 더 많이 부여** 되도록 합니다.

In [43]:
config = {"configurable": {"ensemble_weights": [1, 0]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(metadata={}, page_content='Apple is my favorite company'),
 Document(id='87cd67d4-e126-45d4-9d4c-9987554669e4', metadata={}, page_content='I like apples')]

이번에는 검색시 모든 검색 결과의 가중치가 **FAISS retriever 에 더 많이 부여** 되도록 합니다.

In [44]:
config = {"configurable": {"ensemble_weights": [0, 1]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(id='87cd67d4-e126-45d4-9d4c-9987554669e4', metadata={}, page_content='I like apples'),
 Document(metadata={}, page_content='Apple is my favorite company')]

# 긴 문맥 재정렬(LongContextReorder)

모델의 아키텍처와 상관없이, 10개 이상의 검색된 문서를 포함할 경우 성능이 상당히 저하됩니다.

간단히 말해, 모델이 긴 컨텍스트 중간에 있는 관련 정보에 접근해야 할 때, 제공된 문서를 무시하는 경향이 있습니다.

자세한 내용은 다음 논문을 참조하세요

- https://arxiv.org/abs/2307.03172

이 문제를 피하기 위해, 검색 후 문서의 순서를 재배열하여 성능 저하를 방지할 수 있습니다.


- `Chroma` 벡터 저장소를 사용하여 텍스트 데이터를 저장하고 검색할 수 있는 `retriever`를 생성합니다.
- `retriever`의 `invoke` 메서드를 사용하여 주어진 쿼리에 대해 관련성이 높은 문서를 검색합니다.


In [46]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.3 MB/s et

In [47]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_transformers import LongContextReorder
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


# 임베딩을 가져옵니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

texts = [
    "이건 그냥 내가 아무렇게나 적어본 글입니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]


# 검색기를 생성합니다. (K는 10으로 설정합니다)
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)

검색기에 쿼리를 입력하여 검색을 수행합니다.


In [48]:
query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"

# 관련성 점수에 따라 정렬된 관련 문서를 가져옵니다.
docs = retriever.invoke(query)
docs

[Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.'),
 Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
 Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
 Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),
 Document(metadata={}, page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),
 Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
 Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
 Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
 Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
 Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.')]

`LongContextReorder` 클래스의 인스턴스인 `reordering`을 생성합니다.

- `reordering.transform_documents(docs)`를 호출하여 문서 목록 `docs`를 재정렬합니다.
  - 덜 관련된 문서는 목록의 중간에 위치하고, 더 관련된 문서는 시작과 끝에 위치하도록 재정렬됩니다.


In [49]:
# 문서를 재정렬합니다
# 덜 관련된 문서는 목록의 중간에 위치하고 더 관련된 요소는 시작/끝에 위치합니다.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# 4개의 관련 문서가 시작과 끝에 위치하는지 확인합니다.
reordered_docs

[Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
 Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),
 Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
 Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
 Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'),
 Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
 Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
 Document(metadata={}, page_content='이건 그냥 내가 아무렇게나 적어본 글입니다.'),
 Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
 Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.')]

## Context Reordering을 사용하여 질의-응답 체인 생성


In [50]:
def format_docs(docs):
    return "\n".join([doc.page_content for i, doc in enumerate(docs)])

In [51]:
print(format_docs(docs))

ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
이건 그냥 내가 아무렇게나 적어본 글입니다.
챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.


In [52]:
def format_docs(docs):
    return "\n".join(
        [
            f"[{i}] {doc.page_content} [source: teddylee777@gmail.com]"
            for i, doc in enumerate(docs)
        ]
    )


def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    combined = format_docs(reordered_docs)
    print(combined)
    return combined

재정렬된 문서를 출력합니다.


In [53]:
# 재정렬된 문서를 출력
_ = reorder_documents(docs)

[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: teddylee777@gmail.com]
[2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: teddylee777@gmail.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: teddylee777@gmail.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: teddylee777@gmail.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: teddylee777@gmail.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: teddylee777@gmail.com]
[7] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: teddylee777@gmail.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: teddylee777@gmail.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]


In [54]:
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# 프롬프트 템플릿
template = """Given this text extracts:
{context}

-----
Please answer the following question:
{question}

Answer in the following languages: {language}
"""

# 프롬프트 정의
prompt = ChatPromptTemplate.from_template(template)

# Chain 정의
chain = (
    {
        "context": itemgetter("question")
        | retriever
        | RunnableLambda(reorder_documents),  # 질문을 기반으로 문맥을 검색합니다.
        "question": itemgetter("question"),  # 질문을 추출합니다.
        "language": itemgetter("language"),  # 답변 언어를 추출합니다.
    }
    | prompt  # 프롬프트 템플릿에 값을 전달합니다.
    | ChatOpenAI(model="gpt-4o-mini")  # 언어 모델에 프롬프트를 전달합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
)

`question` 에 쿼리를 입력하고 `language` 에 언어를 입력합니다.

- 재정렬된 문서의 검색 결과도 확인합니다.


In [55]:
answer = chain.invoke(
    {"question": "ChatGPT에 대해 무엇을 말해줄 수 있나요?", "language": "KOREAN"}
)

[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: teddylee777@gmail.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: teddylee777@gmail.com]
[2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: teddylee777@gmail.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: teddylee777@gmail.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: teddylee777@gmail.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: teddylee777@gmail.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: teddylee777@gmail.com]
[7] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: teddylee777@gmail.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: teddylee777@gmail.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: teddylee777@gmail.com]


답변을 출력합니다.


In [56]:
print(answer)

ChatGPT는 OpenAI에 의해 개발된 인공지능 모델로, 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. 이 모델은 지속적으로 개선되고 있으며, 다양한 질문에 답하거나 복잡한 문제를 해결하는 데에도 사용될 수 있습니다. 사용자와 대화하는 형태로 설계되어 있어, 자연스럽고 유용한 대화를 나눌 수 있습니다.


# Parent Document Retriever

**문서 검색과 문서 분할의 균형 잡기**

문서 검색 과정에서 문서를 적절한 크기의 조각(청크)으로 나누는 것은 다음의 **상충되는 두 가지 중요한 요소를 고려**해야 합니다.

1. 작은 문서를 원하는 경우: 이렇게 하면 문서의 임베딩이 그 의미를 가장 정확하게 반영할 수 있습니다. 문서가 너무 길면 임베딩이 의미를 잃어버릴 수 있습니다.
2. 각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우입니다.

**`ParentDocumentRetriever`의 역할**

이 두 요구 사항 사이의 균형을 맞추기 위해 `ParentDocumentRetriever`라는 도구가 사용됩니다. 이 도구는 문서를 작은 조각으로 나누고, 이 조각들을 관리합니다. 검색을 진행할 때는, 먼저 이 작은 조각들을 찾아낸 다음, 이 조각들이 속한 원본 문서(또는 더 큰 조각)의 식별자(ID)를 통해 전체적인 맥락을 파악할 수 있습니다.

여기서 '부모 문서'란, 작은 조각이 나누어진 원본 문서를 말합니다. 이는 전체 문서일 수도 있고, 비교적 큰 다른 조각일 수도 있습니다. 이 방식을 통해 문서의 의미를 정확하게 파악하면서도, 전체적인 맥락을 유지할 수 있게 됩니다.

**정리**

- **문서 간의 계층 구조 활용**: `ParentDocumentRetriever`는 문서 검색의 효율성을 높이기 위해 문서 간의 계층 구조를 활용합니다.
- **검색 성능 향상**: 관련성 높은 문서를 빠르게 찾아내며, 주어진 질문에 대한 가장 적합한 답변을 제공하는 문서를 효과적으로 찾아낼 수 있습니다.
  문서를 검색할 때 자주 발생하는 두 가지 상충되는 요구 사항이 있습니다:


여러 개의 텍스트 파일을 로드하기 위해 `TextLoader` 객체를 생성하고 데이터를 로드합니다.


In [60]:
!pip install langchain_chroma

In [61]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [62]:
loaders = [
    # 파일을 로드합니다.
    TextLoader("/content/appendix-keywords.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(loader.load())

## 전체 문서 검색

이 모드에서는 전체 문서를 검색하고자 합니다. 따라서 `child_splitter` 만 지정하도록 하겠습니다.

- 나중에는 `parent_splitter` 도 지정하여 결과를 비교해 보겠습니다.


In [63]:
# 자식 분할기를 생성합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# DB를 생성합니다.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

`retriever.add_documents(docs, ids=None)` 함수로 문서목록을 추가합니다.

- `ids` 가 `None` 이면 자동으로 생성됩니다.
- `add_to_docstore=False` 로 설정시 document 를 중복으로 추가하지 않습니다. 단, 중복을 체크하기 위한 `ids` 값이 필수 값으로 요구됩니다.


In [64]:
# 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.
retriever.add_documents(docs, ids=None, add_to_docstore=True)

이 코드는 두 개의 키를 반환해야 합니다. 그 이유는 우리가 두 개의 문서를 추가했기 때문입니다.


- `store` 객체의 `yield_keys()` 메서드를 호출하여 반환된 키(key) 값들을 리스트로 변환합니다.


In [65]:
# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

['c6463b1a-2954-4da2-9ead-83c3d9101e88']

이제 벡터 스토어 검색 기능을 호출해 보겠습니다.

우리가 작은 청크(chunk)들을 저장하고 있기 때문에, 검색 결과로 작은 청크들이 반환되는 것을 확인할 수 있을 것입니다.


`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [66]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")

`sub_docs[0].page_content`를 출력합니다.


In [67]:
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


이제 전체 retriever에서 검색해 보겠습니다. 이 과정에서는 작은 청크(chunk)들이 위치한 **문서를 반환** 하기 때문에 상대적으로 큰 문서들이 반환될 것입니다.


`retriever` 객체의 `invoke()` 메서드를 사용하여 쿼리와 관련된 문서를 검색합니다.


In [68]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Word2Vec")

검색된 문서(`retrieved_docs[0]`)의 일부 내용을 출력합니다.


In [69]:
# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 5733


 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을


## 더 큰 Chunk 의 크기를 조절

이전의 결과처럼 **전체 문서가 너무 커서 있는 그대로 검색하기에는 부적합** 할 수 있습니다.

이런 경우, 실제로 우리가 하고 싶은 것은 먼저 원시 문서를 더 큰 청크로 분할한 다음, 더 작은 청크로 분할하는 것입니다.

그런 다음 작은 청크들을 인덱싱하지만, 검색 시에는 더 큰 청크를 검색합니다 (그러나 여전히 전체 문서는 아닙니다).


- `RecursiveCharacterTextSplitter`를 사용하여 부모 문서와 자식 문서를 생성합니다.
  - 부모 문서는 `chunk_size`가 1000으로 설정되어 있습니다.
  - 자식 문서는 `chunk_size`가 200으로 설정되어 있으며, 부모 문서보다 작은 크기로 생성됩니다.


In [70]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# 부모보다 작은 문서를 생성해야 합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장 계층입니다.
store = InMemoryStore()

`ParentDocumentRetriever`를 초기화하는 코드입니다.

- `vectorstore` 매개변수는 문서 벡터를 저장하는 벡터 저장소를 지정합니다.
- `docstore` 매개변수는 문서 데이터를 저장하는 문서 저장소를 지정합니다.
- `child_splitter` 매개변수는 하위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.
- `parent_splitter` 매개변수는 상위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.

`ParentDocumentRetriever`는 계층적 문서 구조를 처리하며, 상위 문서와 하위 문서를 별도로 분할하고 저장합니다. 이를 통해 검색 시 상위 문서와 하위 문서를 효과적으로 활용할 수 있습니다.


In [71]:
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

`retriever` 객체에 `docs`를 추가합니다. `retriever`가 검색할 수 있는 문서 집합에 새로운 문서들을 추가하는 역할을 합니다.


In [72]:
retriever.add_documents(docs)  # 문서를 retriever에 추가합니다.

이제 문서의 수가 훨씬 더 많아진 것을 볼 수 있습니다. 이는 더 큰 청크(chunk)들입니다.


In [73]:
# 저장소에서 키를 생성하고 리스트로 변환한 후 길이를 반환합니다.
len(list(store.yield_keys()))

7

기본 벡터 저장소가 여전히 작은 청크를 검색하는지 확인해 보겠습니다.


`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [74]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


이번에는 `retriever` 객체의 `invoke()` 메서드를 사용하여 문서를 검색합니다.


In [75]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Word2Vec")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
print(retrieved_docs[0].page_content)

정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.
예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.
연관키워드: 딥러닝, 자연어 처리, Attention

HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


# MultiQueryRetriever

거리 기반 벡터 데이터베이스 검색은 고차원 공간에서의 쿼리 임베딩(표현)과 '거리'를 기준으로 유사한 임베딩을 가진 문서를 찾는 방식입니다. 하지만 쿼리의 **세부적인 차이나 임베딩이 데이터의 의미를 제대로 포착하지 못할 경우, 검색 결과가 달라질 수** 있습니다. 또한, 이를 수동으로 조정하는 프롬프트 엔지니어링이나 튜닝 작업은 번거로울 수 있습니다.

이런 문제를 해결하기 위해, `MultiQueryRetriever` 는 주어진 사용자 입력 쿼리에 대해 다양한 관점에서 여러 쿼리를 자동으로 생성하는 LLM(Language Learning Model)을 활용해 프롬프트 튜닝 과정을 자동화합니다.

이 방식은 각각의 쿼리에 대해 관련 문서 집합을 검색하고, 모든 쿼리를 아우르는 고유한 문서들의 합집합을 추출해, 잠재적으로 관련된 더 큰 문서 집합을 얻을 수 있게 해줍니다.

여러 관점에서 동일한 질문을 생성함으로써, `MultiQueryRetriever` 는 거리 기반 검색의 제한을 일정 부분 극복하고, 더욱 풍부한 검색 결과를 제공할 수 있습니다.

In [76]:
# 샘플 벡터DB 구축
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 임베딩 정의
openai_embedding = OpenAIEmbeddings()

# 벡터DB 생성
db = FAISS.from_documents(docs, openai_embedding)

# retriever 생성
retriever = db.as_retriever()

# 문서 검색
query = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = retriever.invoke(query)

# 검색된 문서의 개수 출력
len(relevant_docs)

4

검색된 결과 중 1개 문서의 내용을 출력합니다.


In [77]:
# 1번 문서를 출력합니다.
print(relevant_docs[1].page_content)

가장 강력한 도구로서, Assistant에게 사용자 정의 함수를 지정할 수 있습니다. 이는 Chat Completions API에서의 함수 호출과 매우 유사합니다.


Function calling(함수 호출) 도구를 사용하면 Assistant 에게 사용자 정의 함수 를 설명하여 호출해야 하는 함수를 인자와 함께 지능적으로 반환하도록 할 수 있습니다.


Assistant API는 실행 중에 함수를 호출할 때 실행을 일시 중지하며, 함수 호출 결과를 다시 제공하여 Run 실행을 계속할 수 있습니다. (이는 사용자 피드백을 받아 재게할 수 있는 의미이기도 합니다. 아래 튜토리얼에서 상세히 다룹니다).


## 사용방법

`MultiQueryRetriever` 에 사용할 LLM을 지정하고 질의 생성에 사용하면, retriever가 나머지 작업을 처리합니다.


In [78]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI


# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=db.as_retriever(),
    llm=llm,
)

아래는 다중 쿼리를 생성하는 중간 과정을 디버깅하기 위하여 실행하는 코드입니다.

먼저 `"langchain.retrievers.multi_query"` 로거를 가져옵니다.

이는 `logging.getLogger()` 함수를 사용하여 수행됩니다. 그 다음, 이 로거의 로그 레벨을 `INFO`로 설정하여, `INFO` 레벨 이상의 로그 메시지만 출력되도록 할 수 있습니다.


In [79]:
# 쿼리에 대한 로깅 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

이 코드는 `retriever_from_llm` 객체의 `invoke` 메서드를 사용하여 주어진 `question`과 관련된 문서를 검색합니다.

검색된 문서들은 `unique_docs`라는 변수에 저장되며, 이 변수의 길이를 확인함으로써 검색된 관련 문서의 총 개수를 알 수 있습니다. 이 과정을 통해 사용자의 질문에 대한 관련 정보를 효과적으로 찾아내고 그 양을 파악할 수 있습니다.


In [80]:
# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
# 문서 검색
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['OpenAI Assistant API에서 Functions 기능을 사용하는 방법에 대해 설명해 주세요.', 'OpenAI Assistant API의 Functions를 활용하는 방법에 대한 정보를 제공해 주실 수 있나요?', 'OpenAI Assistant API의 Functions 사용법과 관련된 자료를 찾고 있습니다. 어떤 내용을 참고하면 좋을까요?']


검색된 문서 개수: 5
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


## LCEL Chain 활용하는 방법

- 사용자 정의 프롬프트 정의하고, 정의한 프롬프트와 함께 Chain 을 생성합니다.
- Chain 은 사용자의 질문을 입력 받으면 (아래의 예제에서는) 5개의 질문을 생성한 뒤 `"\n"` 구분자로 구분하여 생성된 5개 질문을 반환합니다.


In [81]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.(5개의 질문을 생성하도록 프롬프트를 작성하였습니다)
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant.
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database.
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION:
{question}

#Answer in Korean:
"""
)

# 언어 모델 인스턴스를 생성합니다.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# LLMChain을 생성합니다.
custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

# 체인을 실행하여 생성된 다중 쿼리를 확인합니다.
multi_queries = custom_multiquery_chain.invoke(question)
# 결과를 확인합니다.(5개 질문 생성)
multi_queries

'OpenAI Assistant API의 Functions 활용 방법에 대해 설명해 주세요.  \nOpenAI Assistant API에서 Functions를 사용하는 절차를 알려주세요.  \nOpenAI Assistant API의 Functions 기능에 대한 안내를 부탁드립니다.  \nOpenAI Assistant API의 Functions 사용에 대한 정보를 제공해 주세요.  \nOpenAI Assistant API에서 Functions를 어떻게 활용할 수 있는지 궁금합니다.  '

이전에 생성한 Chain을 `MultiQueryRetriever` 에 전달하여 retrieve 할 수 있습니다.


In [82]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=custom_multiquery_chain, retriever=db.as_retriever()
)

`MultiQueryRetriever`를 사용하여 문서를 검색하고 결과를 확인합니다.


In [83]:
# 결과
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['OpenAI Assistant API의 Functions 사용법에 대해 설명해 주세요.  ', 'OpenAI Assistant API에서 Functions를 사용하는 방법이 궁금합니다.  ', 'OpenAI Assistant API의 Functions 기능을 어떻게 활용할 수 있나요?  ', 'OpenAI Assistant API의 Functions에 대한 사용 지침을 제공해 주세요.  ', 'OpenAI Assistant API에서 Functions를 사용하는 절차를 알려주세요.']


검색된 문서 개수: 4
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용
